In [ ]:
!pip install transformers[sentencepiece] -q
!pip install datasets -q
!pip install sacrebleu -q
!pip install rouge_score -q
!pip install py7zr -q
!pip install transformers
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
from transformers import pipeline, set_seed
from transformers import BartForConditionalGeneration, BartTokenizer

import pandas as pd
import nltk
from datasets import load_dataset

In [ ]:
# Load your dataset
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Colab Notebooks/IITP_RESEARCH_REDDIT/Dataset_reddit/try.json")

In [ ]:
# Print keys of the DatasetDict object
print(dataset.keys())

# Inspect the structure of a specific dataset within the DatasetDict
print(dataset['train'].features)

dict_keys(['train'])
{'threads': [{'submission_id': Value(dtype='string', id=None), 'subreddit': Value(dtype='string', id=None), 'url': Value(dtype='string', id=None), 'caption': Value(dtype='string', id=None), 'raw_caption': Value(dtype='string', id=None), 'score': Value(dtype='int64', id=None), 'author': Value(dtype='string', id=None), 'created_utc': Value(dtype='int64', id=None), 'permalink': Value(dtype='string', id=None), 'num_comments': Value(dtype='int64', id=None), 'comments_length': Value(dtype='int64', id=None), 'comments': [{'comment_id': Value(dtype='string', id=None), 'body': Value(dtype='string', id=None), 'score': Value(dtype='int64', id=None), 'author': Value(dtype='string', id=None), 'created_utc': Value(dtype='int64', id=None), 'permalink': Value(dtype='string', id=None), 'is_submitter': Value(dtype='bool', id=None), 'chain_length': Value(dtype='int64', id=None), 'replies': [{'comment_id': Value(dtype='string', id=None), 'body': Value(dtype='string', id=None), 'score'

In [ ]:
# For BART
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def summarize_text(model, tokenizer, text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [ ]:
# Iterate over the threads in the dataset
for thread in dataset['train']:
    submission_id = thread['threads'][0]['submission_id']  # Accessing submission_id from the first element in the 'threads' list
    post_caption = thread['threads'][0]['caption']
    print("Post Summary for submission_id:", submission_id)
    post_summary = summarize_text(bart_model, bart_tokenizer, post_caption)
    print("Post Summary:", post_summary)

    # Iterate over the comments in the thread
    for comment in thread['threads'][0]['comments']:
        comment_text = comment['body']
        print("Comment Summary:", summarize_text(bart_model, bart_tokenizer, comment_text))


Post Summary for submission_id: coiccq
Post Summary: summarize: would you put the sectional on the other side?? Would you put it on the side of the couch that's facing the other way? If so, what would you do with it? Would you move it to the side that faces the opposite way?
Comment Summary: I would be changing the side the chaise is on and putting the couch in the other side of the room. The side they are in now are bay windows and I feel I'm losing almost a foot of space. I understand that a tv with windows behind wouldn’t be ideal.
Comment Summary: The chaise part of the section always go along the wall/window side of a room. My sectional couch is non adjustable, so if I move to a new place where the chaise side doesn’t work with the layout of the room, then I will have to get a new couch.
Comment Summary: This sofa won't work in this room no matter what you do so you might as well leave it where it is. Possibly hang the tv on the wall instead of crowding the room with more (dark) f

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Initialize BART tokenizer and model
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large", output_hidden_states=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [ ]:
import torch
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(text1, text2, tokenizer, model):
    inputs = tokenizer([text1, text2], return_tensors='pt', max_length=512, truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    encoder_hidden_states = outputs.encoder_hidden_states
    if encoder_hidden_states is None or len(encoder_hidden_states) == 0:
        raise ValueError("Encoder hidden states are not available in the model output.")
    hidden_states = encoder_hidden_states[-1]  # Extract hidden states from the last layer
    embeddings = hidden_states.mean(dim=1)
    similarity = cosine_similarity(embeddings[0].reshape(1, -1), embeddings[1].reshape(1, -1))
    return similarity.item()

def summarize_comments(post_caption, comments, tokenizer, model):
    relevant_comments = []
    irrelevant_comments = []
    all_comments_summary = ""

    for comment in comments:
        comment_text = comment['body']
        similarity_score = calculate_similarity(post_caption, comment_text, tokenizer, model)
        if similarity_score > 0.5:
            relevant_comments.append(comment_text)
        else:
            irrelevant_comments.append(comment_text)

    all_comments_summary = ' '.join([comment['body'] for comment in comments])
    return all_comments_summary, relevant_comments, irrelevant_comments

# Iterate over the threads in the dataset
for thread in dataset['train']:
    submission_id = thread['threads'][0]['submission_id']  # Accessing submission_id from the first element in the 'threads' list
    post_caption = thread['threads'][0]['caption']
    print("Post Summary for submission_id:", submission_id)
    post_summary = summarize_text(bart_model, bart_tokenizer, post_caption)
    print("Post Summary:", post_summary)

    # Iterate over the comments in the thread
    all_comments_summary, relevant_comments, irrelevant_comments = summarize_comments(post_caption, thread['threads'][0]['comments'], bart_tokenizer, bart_model)

    print("Summary of All Comments:", all_comments_summary)
    print("Relevant Comments:", relevant_comments)
    print("Irrelevant Comments:", irrelevant_comments)


Post Summary for submission_id: coiccq
Post Summary: summarize: would you put the sectional on the other side of the bed?h/t: jimmyjimmy@hotmail.comCommentscommentsPowered by Facebook Comments
Summary of All Comments: So I think I may have made this post confusing. What I would be doing is changing the side the chaise is on and putting the couch in the other side of the room. The side they are in now are bay windows and I feel I’m losing almost a foot of space. I understand that a tv with windows behind wouldn’t be ideal. But I don’t usually watch tv during the day anyway No. In my humble opinion, the chaise part of the section always go along the wall/window side of a room.. because you don’t want the space to feel closed off.  My sectional couch is non adjustable, so if I move to a new place where the chaise side doesn’t work with the layout of the room, then I will have to get a new couch. Absolutely do not put the tv in front of your beautiful windows. This sofa won't work in this 